In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['savefig.transparent'] = False
rcParams['savefig.facecolor'] = 'white'
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from data import load_data, load_rage

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "arial"
sns.set_context("paper")


In [ ]:
df = load_data(rage=False)

def print_opts(d):
    print('\npc name : {}'.format(' '.join(sorted(d['device_name'].unique()))))
    print('model_name : {}'.format(' '.join(sorted(d['model_name'].unique()))))
    print('simulator : {}'.format(' '.join(sorted(d['simulator'].unique()))))
    print('gpu_name : {}'.format(' '.join(sorted(d['gpu_name'].unique()))))
    print('sim_card : {}'.format(' '.join(sorted(d['sim_card'].unique()))))
    
df_rage = load_data(rage=True)
df_rage.head()

In [ ]:

model_names = ['Michaelis-Menten', 'Schlögl','Kinase Cascade',  'EARM 1.0']
print_opts(df)
gpu_only = ['cuda', 'opencl']
gpu_only = df.loc[df.simulator.isin(gpu_only)].copy()
print_opts(gpu_only)

In [ ]:
main_pcs = ['bad', 'aws']
tesla_results = df.loc[df.device_name.isin(main_pcs)].copy()

In [ ]:
print_opts(tesla_results)

In [ ]:
tesla_results.head(10)

# Compare all models on Volta V100

In [ ]:
def compare_times(data, save_name="time_compare", aspect=1.2):
    g = sns.catplot(
        x="n_sim", y="sim_time", hue="simulator", 
        col="model_name", kind="point", data=data, 
        sharey=False, col_wrap=2, height=3, aspect=aspect,
        col_order =model_names,
        hue_order = ['stochkit', 'stochkit_64', 'bng', 'cuda', 'opencl'],
        markers=['o', '*', '+', '<', '^']
    )
    g.set_titles("{col_name}")
    g.set_axis_labels("Number of simulations", "Simulation run time (s)")
    g.set(yscale="log")    
    sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False,
    )
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=45)
    g.set_xticklabels(labels, rotation=45)
    plt.savefig('{}.png'.format(save_name), dpi=300, bbox_inches='tight')
    plt.savefig('{}.pdf'.format(save_name), dpi=300, bbox_inches='tight')
    
compare_times(tesla_results, 'all_models_timing_aspect_1.2', aspect=1.2)
compare_times(tesla_results, 'all_models_timing_aspect_.8', aspect=.8)

In [ ]:
def compare_times_2_models(data, save_name="time_compare"):
    # m_names = ['Michaelis-Menten', 'Schlögl','Kinase Cascade', 'EARM 1.0']
    m_names = ['Michaelis-Menten', 'EARM 1.0']
    d_subset = data.loc[data.model_name.isin(m_names)].copy()
    
    g = sns.catplot(
        x="n_sim", y="sim_time", hue="simulator", 
        col="model_name", kind="point", data=d_subset, 
        sharey=False, col_wrap=1, height=2, aspect=2, 
        col_order =m_names,
        hue_order = ['stochkit', 'stochkit_64', 'bng', 'cuda', 'opencl'],
        markers=['o', '*', '+', '<', '^']
    )
    g.set_titles("{col_name}")
    g.set_axis_labels("Number of simulations", "Simulation run time (s)")
    g.set(yscale="log")    
    sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False,
    )
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=45)
    g.set_xticklabels(labels, rotation=45)
    plt.savefig('{}.png'.format(save_name), dpi=300, bbox_inches='tight')
    plt.savefig('{}.pdf'.format(save_name), dpi=300, bbox_inches='tight')
    
compare_times_2_models(tesla_results, '2_model')

In [ ]:
def plot_ratio(data, save_name="bng_gpu_ratio"):
    """
    Plots the ratio of BNG run times to GPU_SSA run times
    """

    d = pd.pivot_table(
        data[['model_name', 'n_sim', 'simulator', 'sim_time']],
        index=['model_name', 'n_sim'],
        columns='simulator',
        values='sim_time'
    )
    
    def f2(x):
        return '{0:.3f}'.format(x)
    d.to_latex('sum_table_latex.tex', 
               formatters={'bng': f2, 'cuda':f2, 'opencl':f2, 'stochkit':f2, 'stochkit_64':f2},
               bold_rows=True)
    d['ratio_bng'] = d['bng'] / d['cuda']
    d['ratio_stochkit'] = d['stochkit'] / d['cuda']
    d['ratio_stochkit_64'] = d['stochkit_64'] / d['cuda']
#     print(d[['ratio_bng', 'ratio_stochkit', 'ratio_stochkit_64']])
    d.reset_index(inplace=True)
    d.to_csv('all_times.csv')
    
    count = 1
    fig = plt.figure(figsize=(6, 8))
    for i in model_names:
        
        ax = plt.subplot(2,2, count)
        plt.title(i)
        df = d.loc[d.model_name==i][['n_sim', 'ratio_bng', 'ratio_stochkit', 'ratio_stochkit_64']]
        l1,= plt.plot(df['n_sim'], df['ratio_bng'], 'x-r', label='ratio_bng')
        l2,= plt.plot(df['n_sim'], df['ratio_stochkit'], 'o-k', label='ratio_stochkit')
        l3,= plt.plot(df['n_sim'], df['ratio_stochkit_64'], '^-b', label='ratio_stochkit_64')
        ax.set_xscale('log', base=2)    
        ax.set_yscale('log', base=10) 
        plt.ylabel("Simulation time ratio")
        plt.xlabel("Number of simulations")
        if count in (1,2):
            plt.xlabel('')
            plt.xticks([])
        if count in (2, 4):
            plt.ylabel('')
        count+=1
    lgd = fig.legend(
        [l1, l2, l3],
        ["bng/cuda", "stochkit/cuda", "stochkit_64/cuda"],
        fancybox=True, shadow=True, 
        bbox_to_anchor=(1.2, .48),
    )
    plt.subplots_adjust(left=0.07, right=0.93, wspace=0.25, hspace=0.15)
    plt.suptitle('TeslaV1000 Speedup', size=16)
    plt.savefig('{}.png'.format(save_name), dpi=300, bbox_inches='tight', bbox_extra_artists=(lgd,),)
    plt.savefig('{}.pdf'.format(save_name), dpi=300, bbox_inches='tight', bbox_extra_artists=(lgd,),)
plot_ratio(tesla_results)

# GPU scaling

For these tests, I ran the simulations on a 4-GPU rig using the OpenCLSSASimulator. The PCs name was RAGE (thus the load keyword). 

In [ ]:
rage_df = load_data(rage=True)
rage = rage_df.loc[rage_df.precision == 'fp64'].copy()
rage = rage.loc[rage.simulator == 'opencl'].copy()
rage = rage[rage.opencl_args!=''].copy()
rage.head(5)
rage['opencl_args'].unique()
def count_gpus(x):
    return len(x.split(','))

rage['n_gpus'] = rage['opencl_args'].apply(count_gpus)
rage.head(10)


rage_large = rage.loc[rage.n_sim > 2**12].copy()
g = sns.catplot(
    x="n_sim", y="sim_time", hue="n_gpus", 
    col="model_name", kind="point", data=rage_large, 
    sharey=False, col_wrap=2, height=4, 
    col_order=model_names,
    hue_order = [1, 2, 3, 4],
    markers=['o', '*', ',', '<']
);
g.set_titles("{col_name}")
g.set_axis_labels("Number of simulations", "Simulation time(s)")
g.set(yscale="log")    
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)
g.set_xticklabels(labels, rotation=45);

In [ ]:
fig = plt.figure(figsize=(12, 6))
# ax = sns.barplot(x="model_name", y="sim_time", hue="n_gpus", data=rage_large,
#                 estimator=None)
fig.add_subplot(121)
rage_large = rage.loc[rage.n_sim > 2**15].copy()

earm_only = rage_large.loc[rage_large.model_name == 'EARM 1.0'].copy()
g = sns.barplot(
    data=rage_large, 
    x="sim_time", y="model_name", hue="n_gpus",
    ci="sd", palette="dark", alpha=.6, orient='horizonal'
)
g.set(xscale="log") 
plt.title("EARM 1.0 GPU scaling using OpenCL")

fig.add_subplot(122)
pivot_earm_only = pd.pivot_table(earm_only, index='n_sim', values='sim_time', columns=['n_gpus'])
print(pivot_earm_only.head(5))
pivot_earm_only['ratio'] = pivot_earm_only[1]/pivot_earm_only[4]
# print(ratio.head(5))
pivot_earm_only.reset_index(inplace=True)
sns.barplot(data=pivot_earm_only,x='n_sim', y='ratio')
plt.ylabel("Ratio")
plt.title("Timing speedup using 4 GPUs compared to 1")
save_name = 'earm_gpu_scaling'
plt.savefig('{}.png'.format(save_name), dpi=300, bbox_inches='tight', transparent=False)
plt.savefig('{}.pdf'.format(save_name), dpi=300, bbox_inches='tight')
    
    

# Comparing fp64 vs fp32
Compare effect of precision

In [ ]:
# plot fp64 vs fp32

rage = rage_df.loc[rage_df.device_name=='rage'].copy()
rage = rage.loc[rage.simulator.isin(['cuda', 'opencl'])].copy()
rage = rage[rage.opencl_args.isin(['','0:0','0:3'])].copy()
print(rage.head(5))

g = sns.catplot(
        x="n_sim", y="sim_time", hue="precision", 
        col="model_name", row='simulator', kind="point", data=rage, 
        sharey=False, 
#     col_wrap=2, 
    height=4, 
        col_order =model_names,
#         hue_order = ['fp32', 'fp64', ],
        markers=['o', '*',]
    )    

g.set_titles("{col_name}")
g.set_axis_labels("Number of simulations", "Simulation time(s)")
g.set(yscale="log")    
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)
g.set_xticklabels(labels, rotation=45)

# EARM only

In [ ]:
def plot_earm(data, save_name="time_compare"):
    earm_only = data.loc[data.model_name == 'EARM 1.0'].copy()
    earm_only.to_csv('earm_timing.csv')
    
    
    display(earm_only.head(10))
    fig = plt.figure(figsize=(8, 4))

    ax1 = fig.add_subplot(121)
    g = sns.pointplot(
        x="n_sim", y="sim_time", hue="simulator", 
#         col="model_name", 
#         kind="point", 
        data=earm_only, 
#         sharey=False, 
        #col_wrap=1, height=1, 
#         col_order = [ 'EARM 1.0'],
        hue_order = ['stochkit', 'stochkit_64', 'bng', 'cuda', 'opencl'],
        markers=['o', '*', '+', '<', '^'],
        ax=ax1
    )
    plt.title("Simulation times")
    plt.xlabel("Number of simulations")
    plt.ylabel("Simulation time(s)")
    g.set(yscale="log")    
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=45)
    g.set_xticklabels(labels, rotation=45)

    d = pd.pivot_table(
        earm_only[['model_name', 'n_sim', 'simulator', 'sim_time']],
        index=['model_name', 'n_sim'],
        columns='simulator',
        values='sim_time'
    )
    d['ratio_bng'] = d['bng'] / d['cuda']
    d['ratio_stochkit'] = d['stochkit'] / d['cuda']
    d['ratio_stochkit_64'] = d['stochkit_64'] / d['cuda']
    d.reset_index(inplace=True)
    
    ax2 = plt.subplot(1,2, 2)
    
    df = d[['n_sim', 'ratio_bng', 'ratio_stochkit', 'ratio_stochkit_64']]
    
    df = pd.melt(df,
                  id_vars='n_sim', 
                  value_vars=['ratio_bng', 'ratio_stochkit', 'ratio_stochkit_64'],
                  value_name='ratio', 
                 )
    df = df.loc[~df.ratio.isnull()]
    print(df.head(10))
    g = sns.pointplot(
        x="n_sim", y="ratio", hue="simulator", 
#         kind="point",
        data=df, 
#         sharey=False, #col_wrap=1, height=1, 
        palette="Set2",
        markers=['x', 'o', '^'],
        ax=ax2
    )
    g.set(yscale="log")    
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=45)
    g.set_xticklabels(labels, rotation=45)
    plt.title("Simulator timing ratios")
    plt.xlabel("Number of simulations")
    plt.ylabel("Ratio of simulation times")
    plt.subplots_adjust(left=0.07, right=0.93, wspace=0.25, hspace=0.15)
    plt.suptitle('PySB SSA simulator timings for EARM 1.0', size=16)
    plt.savefig('{}.png'.format(save_name), dpi=300, bbox_inches='tight', )
    plt.savefig('{}.pdf'.format(save_name), dpi=300, bbox_inches='tight')
plot_earm(tesla_results, 'earm_only_timing')

In [ ]:
def plot_ratio_cuda_opencl(data, save_name="opencl_cl_ratio"):
    """
    Plots the ratio of BNG run times to GPU_SSA run times
    """
    df_gpu = data.loc[~data.gpu_name.isin(['GTX1060'])].copy()
    df_gpu = df_gpu.loc[~df_gpu.device_name.isin(['rage'])].copy()
#     df_gpu['n_gpus'] = 1
#     def count_gpus(x):
#         return len(x.split(','))
#     df_gpu['n_gpus'] = df_gpu['opencl_args'].apply(count_gpus)
#     print(df_gpu.head(10))
#     df_gpu = df_gpu.loc[~df_gpu.n_gpus==1].copy()
    
    d = pd.pivot_table(
        df_gpu[['model_name', 'n_sim', 'simulator', 'sim_time', 'gpu_name']],
        index=['model_name',  'gpu_name', 'n_sim',],
        columns='simulator',
        values='sim_time'
    )
    
    
    def f2(x):
        return '{0:.3f}'.format(x)
    d.to_latex('sum_table_latex.tex', 
               formatters={'bng': f2, 'cuda':f2, 'opencl':f2, 'stochkit':f2, 'stochkit_64':f2},
               bold_rows=True)
    d['ratio'] = d['opencl'] / d['cuda']
#     print(d[['ratio_bng', 'ratio_stochkit', 'ratio_stochkit_64']])
    d.reset_index(inplace=True)
#     d.to_csv('all_times.csv')
    
    count = 1
    fig = plt.figure(figsize=(8, 9))
    for i in model_names:
        
        ax = plt.subplot(2,2, count)
        plt.title(i)
        df = d.loc[d.model_name==i][['n_sim', 'ratio', 'gpu_name']]
        print(df.gpu_name.unique())
        gpus = [ 'GTX980Ti', 'GTX1080', 'RTX2080', 'RTX3090', 'TeslaK20c', 'TeslaV100']
        legends = []
        labels = []
        for i in gpus:
            dd = df.loc[df.gpu_name==i].copy()
            dd.sort_values('n_sim', inplace=True)
            l1,= plt.plot(dd['n_sim'], dd['ratio'], 'x-', label=i)
            
            legends.append(l1)
            labels.append(i)
        ax.set_xscale('log', base=2)    
#         ax.set_yscale('log', base=10) 
        plt.ylabel("Simulation time ratio")
        plt.xlabel("Number of simulations")
        
        if count in (1,2):
            plt.xlabel('')
            plt.xticks([])
        if count in (2, 4):
            plt.ylabel('')
        count+=1
        lgd = fig.legend(
            legends, labels,
            fancybox=True, shadow=True, 
            bbox_to_anchor=(1.2, .48),
        )
    plt.subplots_adjust(left=0.07, right=0.93, wspace=0.25, hspace=0.15)
    plt.suptitle('OpenCL vs CUDA on Nvidia GPUs', size=16)
    plt.savefig('{}.png'.format(save_name), dpi=300, bbox_inches='tight')# bbox_extra_artists=(lgd,),)
    plt.savefig('{}.pdf'.format(save_name), dpi=300, bbox_inches='tight')#bbox_extra_artists=(lgd,),)
plot_ratio_cuda_opencl(gpu_only)

In [ ]:
def plot_gpu_over_gen(data, save_name="scatter_gpus_over_gen"):
    """
    Plots the ratio of BNG run times to GPU_SSA run times
    """
    df_gpu = data.loc[~data.gpu_name.isin(['GTX1060'])].copy()
    df_gpu = df_gpu.loc[~df_gpu.device_name.isin(['rage'])].copy()

    d = pd.pivot_table(
        df_gpu[['model_name', 'n_sim', 'simulator', 'sim_time', 'gpu_name']],
        index=['model_name',  'gpu_name', 'n_sim',],
        columns='simulator',
        values='sim_time'
    )
    
    d['ratio'] = d['opencl'] / d['cuda']
    d.reset_index(inplace=True)
    
    count = 1
    fig = plt.figure(figsize=(8, 9))
    for i in model_names:
        
        ax = plt.subplot(2,2, count)
        plt.title(i)
        df = d.loc[d.model_name==i][['n_sim', 'cuda', 'gpu_name']]
        print(df.gpu_name.unique())
        gpus = [ 'GTX980Ti', 'GTX1080', 'RTX2080', 
                'RTX3090', 'TeslaV100']
        legends = []
        labels = []
        for i in gpus:
            dd = df.loc[df.gpu_name==i].copy()
            dd.sort_values('n_sim', inplace=True)
            l1,= plt.plot(dd['n_sim'], dd['cuda'], 'x-', label=i)
            
            legends.append(l1)
            labels.append(i)
        ax.set_xscale('log', base=2)    
        ax.set_yscale('log', base=10) 
        plt.ylabel("Simulation time ratio")
        plt.xlabel("Number of simulations")
        
        if count in (1,2):
            plt.xlabel('')
            plt.xticks([])
        if count in (2, 4):
            plt.ylabel('')
        count+=1
        lgd = fig.legend(
            legends, labels,
            fancybox=True, shadow=True, 
            bbox_to_anchor=(1.2, .48),
        )
    plt.subplots_adjust(left=0.07, right=0.93, wspace=0.25, hspace=0.15)
    plt.suptitle('OpenCL vs CUDA on Nvidia GPUs', size=16)
    plt.savefig('{}.png'.format(save_name), dpi=300, bbox_inches='tight')# bbox_extra_artists=(lgd,),)
    plt.savefig('{}.pdf'.format(save_name), dpi=300, bbox_inches='tight')#bbox_extra_artists=(lgd,),)
plot_gpu_over_gen(gpu_only)

In [ ]:
def compare_amd_nvidia():
    
    pal = sns.light_palette("purple", as_cmap=True)
    local_df = load_data()
    print(local_df.sim_card.unique())
    print(local_df.simulator.unique())
    gpus = [
        'cuda', 
        'opencl', 
        'cl_amd_cpu',
        'cl_amd_gpu'
        #'cl_intel_gpu',
        'cl_amd', 'cl_nvidia'
    ]
    
    df_gpu =local_df.loc[local_df.simulator.isin(gpus)].copy()
    
    models = [
        'Michaelis Menten', 
        'Schlögl',
        'JNK Activation Cascade', 
        'EARM 1.0'
    ]
    df_gpu = df_gpu.loc[df_gpu.model_name.isin(models)].copy()

    fig = plt.figure(figsize=(10, 12))

    for n, m in enumerate(models):
        subset = df_gpu.loc[df_gpu['model_name'] == m].copy()

        d = pd.pivot_table(
            subset, index='sim_card', columns='n_sim', values='sim_time',
            fill_value=np.nan,
        )
#         print(d.index.values)
        keep = [
#             'Ryzen_3900x_cl_amd_cpu',
            
            'GTX980Ti_opencl', 
            'GTX980Ti_cuda',
            'GTX1080_opencl', 
            'GTX1080_cuda', 
            'RTX2080_opencl',
            'RTX2080_cuda',
            'RTX3090_opencl',
            'RTX3090_cuda',
#             'TeslaK20c_opencl', 
#             'TeslaK20c_cuda',
            'TeslaV100_opencl', 
            'TeslaV100_cuda',
            'Radeon Vii_opencl',
        ]
    
        d = d.reindex(keep)
        ax = fig.add_subplot(4, 1, n + 1)
        ax.set_title(m)
        if m == 'EARM 1.0':
            fmt = '.0f'
        elif m == 'JNK Activation Cascade':
            fmt = '0.1f'
        else:
            fmt = '.3f'
        sns.heatmap(data=d, cmap=pal, linewidths=0.01, vmin=0, annot=True,
                    fmt=fmt, ax=ax, yticklabels=True)
        plt.ylabel('')
        plt.xlabel("Number of simulations")
    plt.tight_layout()
    plt.subplots_adjust(hspace=.5)
    plt.savefig("compare_amd_nvidia_opencl.png", bbox_inches='tight', dpi=300)
    plt.savefig("compare_amd_nvidia_opencl.pdf", bbox_inches='tight', dpi=300)
    plt.show()


compare_amd_nvidia()

In [ ]:
def compare_cuda_consumer_gpus_all_models():
    pal = sns.light_palette("purple", as_cmap=True)
    local_df = load_data()
    print(local_df.simulator.unique())
    gpus = ['cuda']
    
    df_gpu =local_df.loc[local_df.simulator.isin(gpus)].copy()

    models = [
        'Michaelis Menten', 
        'Schlögl',
        'JNK Activation Cascade', 
        'EARM 1.0'
    ]
    df_gpu = df_gpu.loc[df_gpu.model_name.isin(models)].copy()
    
    fig = plt.figure(figsize=(8, 10))

    for n, m in enumerate(models):
        subset = df_gpu.loc[df_gpu['model_name'] == m].copy()

        d = pd.pivot_table(
            subset, index='sim_card', columns='n_sim', values='sim_time',
            fill_value=np.nan,
        )
        keep = [
            'GTX980Ti_cuda',
            'GTX1080_cuda', 
            'RTX2080_cuda' ,
            'RTX3090_cuda' ,
        ]
        d = d.reindex(keep)
        ax = fig.add_subplot(4, 1, n + 1)
        ax.set_title(m)
        if m == 'EARM 1.0':
            fmt = '.0f'
        elif m == 'JNK Activation Cascade':
            fmt = '0.1f'
        else:
            fmt = '.3f'
        sns.heatmap(data=d, cmap=pal, linewidths=0.01, vmin=0, annot=True,
                    fmt=fmt, ax=ax)

    plt.tight_layout(h_pad=0.5)
    plt.subplots_adjust(hspace=.4)
    plt.savefig("compare_consumer_nvidia_gpus_all_models.png", bbox_inches='tight', dpi=300)
    plt.savefig("compare_consumer_nvidia_gpus_all_models.pdf", bbox_inches='tight', dpi=300)
    plt.show()


compare_cuda_consumer_gpus_all_models()

In [ ]:
def compare_cpus_earm():
    pal = sns.light_palette("purple", as_cmap=True)
    local_df = load_data()
    
    gpus = [
        'cuda', 
        'opencl', 
        'cl_amd_cpu',
        'bng', 
        'stochkit', 
        'stochkit_64',
        
    ]
    df_gpu =local_df.loc[local_df.simulator.isin(gpus)].copy()
    

    models = [
        'EARM 1.0'
    ]
    df_gpu = df_gpu.loc[df_gpu.model_name.isin(models)].copy()
    
    df_gpu.loc[df_gpu.sim_card=='TeslaV100_stochkit', 'sim_card']= 'StochKit'
    df_gpu.loc[df_gpu.sim_card=='TeslaV100_stochkit_64', 'sim_card']= 'StochKit_64'
    df_gpu.loc[df_gpu.sim_card=='TeslaV100_bng', 'sim_card']= 'BNG'
    
    fig = plt.figure(figsize=(9, 2))


    subset = df_gpu.loc[df_gpu['model_name'] == 'EARM 1.0'].copy()

    d = pd.pivot_table(
        subset, index='sim_card', columns='n_sim', values='sim_time',
        fill_value=np.nan,
    )
    keep = [
        'BNG', 'StochKit', 'StochKit_64',
        'Ryzen_3900x_cl_amd_cpu',
    ]
    d = d.reindex(keep)
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title('OpenCL utilized all CPU')

    fmt = '.0f'
    
    sns.heatmap(data=d, cmap=pal, linewidths=0.01, vmin=0, annot=True,
                fmt=fmt, ax=ax)
    plt.yticks(rotation=0) 
    plt.tight_layout(h_pad=0.5)
    plt.subplots_adjust(hspace=.4)
    plt.savefig("compare_consumer_cpus.png", bbox_inches='tight', dpi=300)
    plt.savefig("compare_consumer_cpus.pdf", bbox_inches='tight', dpi=300)
    plt.show()


compare_cpus_earm()

In [ ]:
# consumer NVIDIA GPUs
def compare_cuda_gpus_earm():
    pal = sns.light_palette("purple", as_cmap=True)
    local_df = df.copy()
    print(df.simulator.unique())
    gpus = ['cuda']    
    df_gpu = local_df.loc[local_df.simulator.isin(gpus)].copy()


    models = [
#         'Michaelis Menten', 
#         'Schlögl',
#         'JNK Activation Cascade', 
        'EARM 1.0'
    ]
    df_gpu = df_gpu.loc[df_gpu.model_name.isin(models)].copy()
    df_gpu = df_gpu.loc[~df_gpu.gpu_name.isin(['GTX1060'])]

    fig = plt.figure(figsize=(8, 2))


    subset = df_gpu.loc[df_gpu['model_name'] == 'EARM 1.0'].copy()

    d = pd.pivot_table(
        subset, index='sim_card', columns='n_sim', values='sim_time',
        fill_value=np.nan,
    )
    print(d.index.values)
    keep = [
        'GTX980Ti_cuda',
        'GTX1080_cuda', 
        'RTX2080_cuda' ,
        'RTX3090_cuda' ,
#             'TeslaK20c_opencl', 
#             'TeslaK20c_cuda',
#             'TeslaV100_opencl', 
        'TeslaV100_cuda',
#             'Radeon Vii_opencl',
    ]
    d = d.reindex(keep)
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title('Performance over generations for EARM')

    fmt = '.0f'
    
    sns.heatmap(data=d, cmap=pal, linewidths=0.01, vmin=0, annot=True,
                fmt=fmt, ax=ax)

    plt.tight_layout(h_pad=0.5)
    plt.subplots_adjust(hspace=.4)
    plt.savefig("compare_consumer_nvidia_gpus.png", bbox_inches='tight', dpi=300)
    plt.savefig("compare_consumer_nvidia_gpus.pdf", bbox_inches='tight', dpi=300)
    plt.show()


compare_cuda_gpus_earm()

## Creates plot of all timings for CUDA and CPUs

In [ ]:
def summary_of_all():
    pal = sns.light_palette("purple", as_cmap=True)
    local_df = df.copy()
    gpus = [
        'cuda', 
        'opencl', 
        'cl_amd_cpu',
        'bng', 
        'stochkit', 
        'stochkit_64',
        
    ]
    
    df_gpu = local_df.loc[local_df.simulator.isin(gpus)].copy()
    df_gpu.loc[df_gpu.sim_card=='TeslaV100_stochkit', 'sim_card']= 'StochKit'
    df_gpu.loc[df_gpu.sim_card=='TeslaV100_stochkit_64', 'sim_card']= 'StochKit_64'
    df_gpu.loc[df_gpu.sim_card=='TeslaV100_bng', 'sim_card']= 'BNG'
    models = [
        'Michaelis Menten', 
        'Schlögl',
        'JNK Activation Cascade', 
        'EARM 1.0'
    ]
    df_gpu = df_gpu.loc[df_gpu.model_name.isin(models)].copy()
    
    fig = plt.figure(figsize=(12, 18))

    for n, m in enumerate(models):
        subset = df_gpu.loc[df_gpu['model_name'] == m].copy()
    

        d = pd.pivot_table(
            subset, index='sim_card', columns='n_sim', values='sim_time',
            fill_value=np.nan,
        )
        keep = [
            'BNG', 'StochKit', 'StochKit_64',
            'Ryzen_3900x_cl_amd_cpu',
            'GTX980Ti_cuda',
            'GTX1080_cuda', 
            'RTX2080_opencl' ,
            'RTX2080_cuda' ,
            'RTX3090_cuda',
            'RTX3090_opencl',
            'Radeon Vii_opencl',
            'TeslaV100_cuda',
            'TeslaV100_opencl'
        ]
        d = d.reindex(keep)
        ax = fig.add_subplot(5, 1, n + 1)
        ax.set_title(m)
        if m == 'EARM 1.0':
            fmt = '.0f'
        elif m == 'JNK Activation Cascade':
            fmt = '0.1f'
        else:
            fmt = '.3f'
        sns.heatmap(data=d, cmap=pal, linewidths=0.01, vmin=0, annot=True,
                    fmt=fmt, ax=ax)

    plt.tight_layout()
    plt.savefig("compare_gpus.png", bbox_inches='tight', dpi=300)
    plt.show()


summary_of_all()